### Import Libraries

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt


### Define Configs

In [ ]:
tickers = {
    'MSFT': {'ipo': '1986-03-13'},
    'AAPL': {'ipo': '1980-12-12'},
    'NVDA': {'ipo': '1999-01-22'},
    'AMZN': {'ipo': '1997-05-15'},
    'GOOG': {'ipo': '2004-08-19'},
    'META': {'ipo': '2012-05-18'},
    'TSLA': {'ipo': '2010-06-29'}
}

initial_capital = 1_000_000
commission = 0.001  # 0.10%
slippage = 0.0002  # 0.02%
min_shares = 10
risk_free_rate = 0.02
trading_days = 252  # Annual trading days
start_date = "1981-01-01"
end_date = "2023-12-31"


### Fetching data

In [ ]:
def fetch_data(tickers):
    data = {}
    for ticker, details in tickers.items():
        ipo_date = details['ipo']
        print(f"Fetching data for {ticker} (IPO: {ipo_date})...")

        df = yf.download(ticker, start=ipo_date, end=end_date, auto_adjust=True)

        if df.empty:
            print(f"Warning: No data for {ticker}.")
            continue

        df['Ticker'] = ticker
        data[ticker] = df
    return data

data = fetch_data(tickers)


### Compute RSI

In [ ]:
def compute_rsi(series, window=14):
    delta = series.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window, min_periods=1).mean()
    avg_loss = loss.rolling(window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

for ticker in data:
    df = data[ticker]
    df['RSI'] = compute_rsi(df['Close'])


In [ ]:
signals = {}
for ticker in data:
    df = data[ticker]
    df['signal'] = np.where(df['RSI'] < 25, 'buy', np.where(df['RSI'] > 75, 'sell', None))
    df['position'] = None
    signals[ticker] = df[['signal', 'position']]


In [ ]:
all_dates = pd.date_range(start=start_date, end=end_date)
signals_df = pd.DataFrame(index=all_dates)

for ticker in tickers:
    if ticker in signals:
        signals_df[ticker] = signals[ticker]['Signal']

positions_df = signals_df.copy()
positions_df[:] = None  # Start with no positions

for ticker in tickers:
    if ticker in signals:
        prev_position = None
        for date in signals_df.index:
            current_signal = signals_df.at[date, ticker] if date in signals_df.index else None

            if current_signal == 'buy':
                prev_position = 1  # Long position
            elif current_signal == 'sell':
                prev_position = 0  # Exit position
            else:
                prev_position = prev_position  # Maintain previous position

            positions_df.at[date, ticker] = prev_position


In [ ]:
# TODO: Backtest